# 함수로 만들기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np

## 1. chrome 브라우저 열기

In [2]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")
#assert "Google" in driver.title


time.sleep(2)


element = driver.find_element_by_name('q')


#element = WebDriverWait(driver, 10).until(lambda x: x.driver.find_element_by_name('q'))
        

#delay = 10
#try:
#    element=WebDriverWait(driver, delay).until(EC.presence_of_element_located(driver.find_element_by_name('q')))
#except TimeoutException:
#    print ("Loading took too much time!-Try again")


element.clear()
element.send_keys('google map')
element.submit()


#google map site 들어가기
element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

## 2. data crawrling

In [262]:
# 여러개의  shop 목록에서 주소로 찾기
def find_shop_by_address(select_shop_html, shop_address):
    soup = BeautifulSoup(select_shop_html, 'html.parser')
    all_address =soup.find_all('span', attrs = {'class': "section-result-location"} )
    shop_index = 0
    shop_select_dict = {}
    for i, address in enumerate(all_address):
        print(address.text)
        gu = shop_address[0] in address.text
        dong = shop_address[1] in address.text
        bungi = shop_address[2] in address.text
        if gu and dong and bungi:
            shop_select_dict[str(i+1)] = 'match'
        elif gu or dong:
            shop_select_dict[str(i+1)] = 'not match'
        else:
            pass
#     print(shop_select_dict)
    return shop_select_dict


In [263]:
# 주소와 total 별점 가져오기
def get_totalScore(html,shop_address):
    gu = shop_address[0]
    dong = shop_address[1] 
    bungi = shop_address[2]
    
    soup = BeautifulSoup(html, 'html.parser')

    infos =soup.find_all('span',attrs = {'class': "section-info-text"} )
    
    address =infos[0].text
#     print(address)

    if gu in address and dong in address and bungi in address:
        match = 'match'
    else:
        match = 'not match'

    
    total_score = soup.find_all('div',attrs = {'class': "gm2-display-2"} )

    if len(total_score)!=0:
        total_score = total_score[0].text
    else:
        total_score = np.nan
    
    return address, total_score, match

In [264]:
def get_info(html, shop_address):
# shop 주소, 총 별점 갖고오기
    address, total_score, match =get_totalScore(html,shop_address)
    
#     print('total_score',total_score)
    # 리뷰가 없는 경우
    if  np.isnan(total_score):
        review_list = np.nan
        score_list= np.nan
        

    # 리뷰가 있는 경우
    else:
        # 리뷰 text(버튼 )클릭
        review_button = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
        review_button.click()

        time.sleep(2)

        # 스크롤 다운
        try:
            element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
            element.click()

        except Exception as err:
            print(err)

        i = 0
        while  True:
            i += 1
            html = driver.find_element_by_tag_name('html')
            time.sleep(SCROLL_PAUSE_TIME)

            text1 = html.text
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.find_element_by_tag_name('html')
            text2 = html.text

            if text1 == text2:
                break
            else:
                continue

        review_html = driver.page_source

        review_list, score_list=get_review(review_html)    

    return address, total_score, review_list, score_list, match

In [265]:
def get_review(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 개별 리뷰
    total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )

    review_list=[]
    for i, review in enumerate(total_review):
        review_list.append(review.text)

    #### 개별 별점
    scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )

    score_list = []
    for score in scores:
        score_list.append(score.attrs['aria-label'])

    return review_list, score_list

In [266]:
def srart_crwarling(forpet_hash,shop_name, shop_address):
    print(shop_name, shop_address)

    # shop 이름 입력
    element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
    element.clear()
    
    search_word = shop_name + ' ' +shop_address[0]
    print(search_word)
    element.send_keys(search_word)
    element.submit()
    
    search_button = driver.find_element_by_xpath('//button[@class="searchbox-searchbutton"]')
    search_button.click()
    
    time.sleep(SCROLL_PAUSE_TIME)

    # 키워드 입력 후, dict로 인덱스와 match 여부 받아오기
    select_shop_html = driver.page_source
    shop_select_dict= find_shop_by_address(select_shop_html, shop_address)
    
    # shoh이 여러개인 경우
    if len(shop_select_dict) != 0:
        print(f'shop이 {len(shop_select_dict)}개입니다')

        for index, match in shop_select_dict.items():
            shop_index =  2 * int(index) -1
#             print('shop_index', shop_index)
            match = match

            time.sleep(SCROLL_PAUSE_TIME)

            select_shop = driver.find_element_by_xpath(f'//div[@class = "section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical"]/div/div[{shop_index}]/div[1]/div[1]')
            
            select_shop.click()

            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.page_source

            address, total_score, review_list, score_list, match =get_info(html, shop_address)
            print(address, total_score, review_list, score_list, match )
            
            if match == 'match':
                total_table_insert(match_total_table, forpet_hash, shop_name, address, total_score, match)
                review_table_insert(match_review_table, forpet_hash, shop_name, review_list, score_list, match)
            else:
                total_table_insert(not_match_total_table, forpet_hash, shop_name, address, total_score, match)
                review_table_insert(not_match_review_table, forpet_hash, shop_name, review_list, score_list, match)

              
            time.sleep(SCROLL_PAUSE_TIME)
            driver.back()  


    # shop이 1개 인경우
    else:
        print('shop이 1개입니다')
        html = driver.page_source

        address, total_score, review_list, score_list, match =get_info(html, shop_address)

        print(address, total_score, review_list, score_list, match )
        
        if match == 'match':
                total_table_insert(match_total_table, forpet_hash, shop_name, address, total_score, match)
                review_table_insert(match_review_table, forpet_hash, shop_name, review_list, score_list, match)
        else:
            total_table_insert(not_match_total_table, forpet_hash, shop_name, address, total_score, match)
            review_table_insert(not_match_review_table, forpet_hash, shop_name, review_list, score_list, match)

    driver.back()  
     
#     try:
#         back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
#         back_button.click()
#     except:
#         pass
        
    

## 3. Dataframe

### 3.1 total_table

In [229]:
not_match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'total_score', 'match'])
not_match_total_table

,forpet_hash,shop_name,shop_address,total_score,match


In [230]:
match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'total_score', 'match'])
match_total_table

,forpet_hash,shop_name,shop_address,total_score,match


In [231]:
def total_table_insert(total_table, forpet_hash, shop_name, address, total_score, match):
    print(len(total_table))
    if total_score != None:
        total_table.loc[len(total_table)+1] = [forpet_hash, shop_name, address, total_score, match]
    else:
        total_table.loc[len(total_table)+1] = [forpet_hash, shop_name, address,np.nan, match]
    return total_table

### 3.2. review table

In [232]:
not_match_review_table= pd.DataFrame(columns = ['forpet_hash', 'num', 'review', 'score', 'match'])
not_match_total_table

,forpet_hash,shop_name,shop_address,total_score,match


In [233]:
match_review_table= pd.DataFrame(columns = ['forpet_hash', 'num', 'review', 'score', 'match'])
match_review_table

,forpet_hash,num,review,score,match


In [254]:
def review_table_insert(review_table, forpet_hash, shop_name, review_list, score_list, match):
    print(len(review_table))
    if np.isnan(review_list):
        review_table.loc[1+len(review_table)] = [forpet_hash, 1, np.nan, np.nan , match]
    else:
        for i, (review, score) in enumerate(zip(review_list, score_list)):
            review_table.loc[i+1+len(review_table)] = [forpet_hash, i, review, score, match]   
    
    return review_table

## 4. 실행

In [259]:
forpet_hashes = ['4b793eb834462b029e435eaeb49433efc61755c9e15a24',
     'f5169c15a126e4059f0a89bb80fccd236693d5056d24ae',
     'aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb',
     '2806f173e3a783ed14d74c9a83da2f62c34f45d470e893',
     'aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c']

address_names = ['서울 종로구 평창동 530-8',
              '서울 종로구 창신동 468-1',
              '서울 종로구 창신동 436-64',
              '서울 종로구 홍지동 103-24',
              '서울 종로구 필운동 137-5']

place_names  = ['쿠나이앤티',
               '애견용품할인매장',
               '펫클럽 청계점',
               '나니스펫푸드',
               '서촌양품']

In [260]:
num = 1
forpet_hash = forpet_hashes[num]
shop_address = address_names[num].split(' ')[1:]
shop_name = place_names[num]
print(shop_address, shop_name)

['종로구', '창신동', '468-1'] 애견용품할인매장


In [261]:
SCROLL_PAUSE_TIME = 2

srart_crwarling(forpet_hash,shop_name,shop_address)

print('match_total_table\n',match_total_table)
print('not_match_total_table\n',not_match_total_table)
print('match_review_table\n',match_review_table)
print('not_match_review_table\n',not_match_review_table)

애견용품할인매장 ['종로구', '창신동', '468-1']
애견용품할인매장 종로구
창신동 315-1번지 1층 종로구 서울특별시 KR
창신동 436-64번지
금호동1가 513번지 성동구 서울특별시 KR
{'1': 'not match', '2': 'not match'}
shop이 2개입니다
shop_index 1
창신동 315-1번지 1층 종로구 서울특별시 KR
total_score nan
창신동 315-1번지 1층 종로구 서울특별시 KR nan nan nan not match
6
4
shop_index 3
서울특별시 종로구 창신동 436-64번지
total_score nan
서울특별시 종로구 창신동 436-64번지 nan nan nan not match
7
5
match_total_table
                                       forpet_hash shop_name  \
1  aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c  영등포 동물병원   

           shop_address total_score  match  
1  서울특별시 영등포구 당산동1가 348         3.8  match  
not_match_total_table
                                       forpet_hash shop_name  \
1  4b793eb834462b029e435eaeb49433efc61755c9e15a24     쿠나이앤티   
2  4b793eb834462b029e435eaeb49433efc61755c9e15a24     쿠나이앤티   
3  4b793eb834462b029e435eaeb49433efc61755c9e15a24     쿠나이앤티   
4  4b793eb834462b029e435eaeb49433efc61755c9e15a24     쿠나이앤티   
5  f5169c15a126e4059f0a89bb80fccd236693d5056d24ae  애견

In [215]:
shop_address = ['영등포구', '당산동1가', '348']
shop_name = '영등포 동물병원'

> x-path 조절하기
- http://blog.naver.com/PostView.nhn?blogId=kiddwannabe&logNo=221323738780&categoryNo=35&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

> 스크롤 조절
https://m.blog.naver.com/kiddwannabe/221310063239